In [12]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot = True)

n_classes = 10 
batch_size = 128

x = tf.placeholder('float', [None, 784])
y = tf.placeholder('float')

def conv2d(x, weights):
    # strides [1, 1, 1, 1] => move by one pixel at a time in each directions
    # padding='SAME' => when you arrive at the end of the image and the window
    #                   "falls" out then you pad the image by repeating the 
    #                   pixels.
    return tf.nn.conv2d(x, weights, strides=[1, 1, 1, 1], padding='SAME') 

def maxpool2d(x):
    # ksizes [1, 2, 2, 1] => size of the window (2x2 pooling)
    # strides [1, 2, 2, 1] => move the pooling by 2 pixels at a time (no overlap while moving)
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME') # strides [1, 2, 2, 1] => move by one pixel at a time in each directions

def convolutional_neural_network(x):
    weights = {'w_conv1':tf.Variable(tf.random_normal([5, 5, 1, 32])), #5x5 convolution, 1 input to 32 outputs 
               'w_conv2':tf.Variable(tf.random_normal([5, 5, 32, 64])), # \\, 32 in 64 output (features)
               'w_fc':tf.Variable(tf.random_normal([7*7*64, 1024])), # 7*7 inputs with 64 features, 1024 nodes 
               'out':tf.Variable(tf.random_normal([1024, n_classes]))}

    # just for the output
    biases = {'b_conv1':tf.Variable(tf.random_normal([32])), 
              'b_conv2':tf.Variable(tf.random_normal([64])),
              'b_fc':tf.Variable(tf.random_normal([1024])), 
              'out':tf.Variable(tf.random_normal([n_classes]))}
    
    # reashape a 764 pixel image to a flat 28x28 image
    x = tf.reshape(x, shape=[-1, 28, 28, 1])
    
    # firs hidden connected layer
    conv1 = conv2d(x, weights['w_conv1'])
    conv1 = maxpool2d(conv1)
    # second hidden connected layer
    conv2 = conv2d(conv1, weights['w_conv2'])
    conv2 = maxpool2d(conv2)
    # fully connected layer
    fc = tf.reshape(conv2, [-1, 7*7*64])
    fc = tf.nn.relu(tf.matmul(fc, weights['w_fc'])+biases['b_fc'])
    
    # output layer
    output = tf.matmul(fc, weights['out']+biases['out'])
    return output

def train_neural_network(x):
    prediction = convolutional_neural_network(x)
    # OLD VERSION:
    #cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(prediction,y) )
    # NEW:
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y) )
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    hm_epochs = 10
    with tf.Session() as sess:
        # OLD:
        #sess.run(tf.initialize_all_variables())
        # NEW:
        sess.run(tf.global_variables_initializer())

        for epoch in range(hm_epochs):
            epoch_loss = 0
            for _ in range(int(mnist.train.num_examples/batch_size)):
                epoch_x, epoch_y = mnist.train.next_batch(batch_size)
                _, c = sess.run([optimizer, cost], feed_dict={x: epoch_x, y: epoch_y})
                epoch_loss += c

            print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',epoch_loss)

        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))

        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:',accuracy.eval({x:mnist.test.images, y:mnist.test.labels}))

train_neural_network(x)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Epoch 0 completed out of 10 loss: 40445884.9219
Epoch 1 completed out of 10 loss: 106636.275915
Epoch 2 completed out of 10 loss: 30345.2316298
Epoch 3 completed out of 10 loss: 13575.4600334


KeyboardInterrupt: 